# Estival/PyMC

In this notebook, we will build a BayesianCompartmentalModel, and calibrate it using PyMC

In [ ]:
# Uncomment to install in colab
#!pip install estival==0.2.6

In [ ]:
# This is required for parallel evaluation in notebooks
# Note that if running under (non-WSL) Windows, you should
# disable this line, and use single threaded evaluation in pymc

import multiprocessing as mp
mp.set_start_method('forkserver')

In [ ]:
import summer2
import numpy as np
import pandas as pd

In [ ]:
from summer2.extras import test_models

In [ ]:
m = test_models.sir()

In [ ]:
defp = m.get_default_parameters()
defp

In [ ]:
m.run({"contact_rate": 0.5, "recovery_rate": 0.4})
do_def = m.get_derived_outputs_df()
obs_clean = do_def["incidence"].iloc[0:50]
obs_noisy = obs_clean * np.exp(np.random.normal(0.0,0.2,len(obs_clean)))
obs_clean.plot()
obs_noisy.plot(style='.')

In [ ]:
# The following imports are the 'building blocks' of estival models

# Targets represent data we are trying to fit to
from estival import targets as est

# We specify parameters using (Bayesian) priors
from estival import priors as esp

# Finally we combine these with our summer2 model in a BayesianCompartmentalModel (BCM)
from estival.model import BayesianCompartmentalModel

In [ ]:
# Specify a Truncated normal target with a free dispersion parameter
targets = [
    est.TruncatedNormalTarget("incidence", obs_noisy, (0.0,np.inf),
        esp.UniformPrior("incidence_dispersion",(0.1, obs_noisy.max()*0.1)))
]

In [ ]:
# Uniform priors over our 2 model parameters
priors = [
    esp.UniformPrior("contact_rate", (0.01,1.0)),
    esp.TruncNormalPrior("recovery_rate", 0.5, 0.2, (0.01,1.0)),
]

In [ ]:
# The BayesianCompartmentalModel class is the primary entry point to all optimization and calibration
# methods in estival
# It takes a CompartmentalModel object, default parameters, priors, and targets
# The default parameters will be used as fixed values when no prior is specified for a given parameter

bcm = BayesianCompartmentalModel(m, defp, priors, targets)

In [ ]:
from estival.calibration import pymc as epm

In [ ]:
import pymc as pm

In [ ]:
with pm.Model() as model:
    
    # This is all you need - a single call to use_model
    # include_ll will optionally output the BCM loglikelihood as a sampled value
    # It is recommended this value be left at the default (False),
    # but we include it in this notebook for convenience

    variables = epm.use_model(bcm, include_ll=True)
    
    # The log-posterior value can also be output, but may incur additional overhead
    # Use jacobian=False to get the unwarped value (ie just the 'native' density of the priors
    # without transformation correction factors)
    # pm.Deterministic("logp", model.logp(jacobian=False))
    
    # Now call a sampler using the variables from use_model
    # In this case we use the Differential Evolution Metropolis sampler
    # See the PyMC docs for more details
    idata_mh = pm.sample(step=[pm.DEMetropolis(variables)], draws=4000, tune=1000,cores=4,chains=4)

## Using arviz to examine outputs

In [ ]:
import arviz as az

In [ ]:
az.summary(idata_mh)

In [ ]:
# Optional - select some subset out of the resulting trace - useful if
# you require additional burnin
# subset = idata_mh.sel(draw=slice(500, None), groups="posterior")

In [ ]:
az.plot_trace(idata_mh, figsize=(16,3.2*len(idata_mh.posterior)),compact=False);#, lines=[("m", {}, mtrue), ("c", {}, ctrue)]);

In [ ]:
az.plot_posterior(idata_mh);

In [ ]:
# If we have included the loglikelihood in our outputs, then obtaining the MLE is simple
# Note that it is generally far more efficient to generate the loglikelihood as a
# separate step, but we follow the 'easy path' in this notebook for clarity

caldf = idata_mh.to_dataframe(groups="posterior")
ll_sorted = caldf.sort_values(by="loglike", ascending=False)

In [ ]:
ll_sorted

In [ ]:
# Simply get the first item from our sorted Dataframe

mle_params = ll_sorted.iloc[0][list(bcm.priors)].to_dict()
mle_params

In [ ]:
# As you can see, this is exactly the value output from the original BCM
bcm.loglikelihood(**mle_params), ll_sorted.iloc[0]["loglike"]

In [ ]:
# Run the model with these parameters
mle_res = bcm.run(mle_params)

In [ ]:
# ...and plot some results
variable = "incidence"

pd.Series(mle_res.derived_outputs[variable]).plot(title = f"{variable} (MLE)")
bcm.targets[variable].data.plot(style='.');

In [ ]:
# Do some basic uncertainty sampling

In [ ]:
def sample_outputs(bcm, idata, n_samples):
    caldf = idata.to_dataframe(groups="posterior")
    caldf = caldf[list(bcm.priors)]
    draws = caldf.index
    sel_draws = np.random.choice(draws, n_samples, False)
    samples = caldf.loc[sel_draws]
    
    all_res = []
    for k,v in samples.iterrows():
        cur_sample = v.to_dict()
        all_res.append(bcm.run(cur_sample))
        
    return all_res

In [ ]:
# Play around with n_samples to get a sense of what is required to accurately represent the posterior
all_res = sample_outputs(bcm, idata_mh, 400)

In [ ]:
def get_uncertainty_quantiles(all_res, quantiles, variable):
    stacked_incidence = np.stack([res.derived_outputs[variable] for res in all_res])
    return pd.DataFrame(np.quantile(stacked_incidence, quantiles, axis=0).T, columns=quantiles)

In [ ]:
variable = "incidence"
quantiles = (0.025,0.25,0.5,0.75,0.975)

udf = get_uncertainty_quantiles(all_res, quantiles, variable)

fig = udf.plot(title=variable,alpha=0.7)
pd.Series(mle_res.derived_outputs[variable]).plot(style='--')
bcm.targets[variable].data.plot(style='.',color="black", ms=3, alpha=0.8);